In [1]:
import random
import io
import tensorflow as tf
from PIL import Image
from tqdm import tqdm_notebook
random.seed(12345)
from pathlib import Path
import tensorshow

In [2]:
tf.__version__

'1.13.1'

In [3]:
IMAGENET_TRAIN_IMGS = Path('/home/joel/imagenet/ILSVRC2012_img_train')
IMAGENET_VAL_IMGS = Path('/home/joel/imagenet/ILSVRC2012_img_val')
IMAGENET_TRAIN_LABELS = Path('/home/joel/imagenet/ILSVRC2012_bbox_train_v2')
IMAGENET_VAL_LABELS = Path('/home/joel/imagenet/ILSVRC2012_bbox_val_v3/val')
code_to_labels_file = Path('/home/joel/imagenet/LOC_synset_mapping.txt')

In [31]:
imagenet_train_dirs = [x for x in IMAGENET_TRAIN_IMGS.iterdir() if x.is_dir()]
imagenet_filenames = [file for file in IMAGENET_VAL_IMGS.iterdir()]

In [32]:
random.shuffle(imagenet_filenames)

In [33]:
len(imagenet_filenames)

50000

In [34]:
imagenet_filenames[:1]

[PosixPath('/home/joel/imagenet/ILSVRC2012_img_val/ILSVRC2012_val_00023106.JPEG')]

In [8]:
# Remove the png file hiding in there.
imagenet_filenames.remove(Path('/home/joel/imagenet/ILSVRC2012_img_train/n02105855/n02105855_2933.JPEG'))

In [9]:
len(imagenet_filenames)

1281166

In [36]:
sharded_filenames = []
shard_size = len(imagenet_filenames) // 128 + 1
assert (shard_size * 128 >= len(imagenet_filenames) and (shard_size-1) * 128 < len(imagenet_filenames))

In [37]:
for i in range(0, len(imagenet_filenames), shard_size):
    sharded_filenames.append(imagenet_filenames[i:i+shard_size])

In [38]:
len(sharded_filenames)

128

In [39]:
sum(len(x) for x in sharded_filenames)

50000

In [14]:
code_to_labels = {}
with code_to_labels_file.open() as f:
    for i, line in enumerate(f):
        code = line[:9]
        desc = line[9:].strip().split(',')[0]
        code_to_labels[code] = (i, desc)

In [69]:
len(code_to_labels)
print(code_to_labels)

{'n01440764': (0, 'tench'), 'n01443537': (1, 'goldfish'), 'n01484850': (2, 'great white shark'), 'n01491361': (3, 'tiger shark'), 'n01494475': (4, 'hammerhead'), 'n01496331': (5, 'electric ray'), 'n01498041': (6, 'stingray'), 'n01514668': (7, 'cock'), 'n01514859': (8, 'hen'), 'n01518878': (9, 'ostrich'), 'n01530575': (10, 'brambling'), 'n01531178': (11, 'goldfinch'), 'n01532829': (12, 'house finch'), 'n01534433': (13, 'junco'), 'n01537544': (14, 'indigo bunting'), 'n01558993': (15, 'robin'), 'n01560419': (16, 'bulbul'), 'n01580077': (17, 'jay'), 'n01582220': (18, 'magpie'), 'n01592084': (19, 'chickadee'), 'n01601694': (20, 'water ouzel'), 'n01608432': (21, 'kite'), 'n01614925': (22, 'bald eagle'), 'n01616318': (23, 'vulture'), 'n01622779': (24, 'great grey owl'), 'n01629819': (25, 'European fire salamander'), 'n01630670': (26, 'common newt'), 'n01631663': (27, 'eft'), 'n01632458': (28, 'spotted salamander'), 'n01632777': (29, 'axolotl'), 'n01641577': (30, 'bullfrog'), 'n01644373': (31,

In [41]:
def get_image_bytes(filename):
    im = Image.open(filename)
    x, y = im.size
    if x > y:
        cropped = im.crop(((x-y) // 2, 0,(x+y) // 2, y))
    elif x < y:
        cropped = im.crop((0, (y-x) // 2, x, (y+x) // 2))
    else:
        cropped = im
    
    im = cropped.resize((256, 256), Image.BICUBIC)
    im_bytes = io.BytesIO()
    im.save(im_bytes, format='JPEG')
    return im_bytes.getvalue()

def to_example(encoded, filename, label, synset, text):
    feature_dict = {
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename])),
        'image/class/label' : tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
        'image/class/synset' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[synset])),
        'image/class/text' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[text])),
    }
    
    return tf.train.Example(features=tf.train.Features(feature=feature_dict))

train_tfrecord_path = '/home/joel/imagenet/tfrecords/val-{:04}-0128'
for i, names in tqdm_notebook(enumerate(sharded_filenames), desc='shards'):
    writer = tf.python_io.TFRecordWriter(train_tfrecord_path.format(i))
    for name in tqdm_notebook(names, desc='shard {}'.format(i)):
        encoded = get_image_bytes(name)
        filename = bytes(name.parts[-1], 'utf-8')
        synset = 'n' + name.parts[-1][15:-5]
        label, text = code_to_labels[synset]
        synset = bytes(synset, 'utf-8')
        text = bytes(text, 'utf-8')
        writer.write(to_example(encoded, filename, label, synset, text).SerializeToString())
    
    writer.close()
    

KeyError: 'n00023106'

In [22]:
corrupt = sharded_filenames[468]
for name in corrupt:
    im = Image.open(name)
    exif_data = im._getexif()

In [ ]:
df = tensorshow.html_file_from('/home/joel/imagenet/tfrecords/train-0000-1024', 'tfrecord.html', limit=100)

In [ ]:
im = Image.open('/home/joel/imagenet/ILSVRC2012_img_train/n02105855/n02105855_321.JPEG')

In [ ]:
x, y = im.size

In [ ]:
min_dim = min(x, y)
256/min_dim
im.show()

In [ ]:
if x > y:
    cropped = im.crop(((x-y) // 2, 0,(x+y) // 2, y))
    cropped.show()

In [ ]:
resized = cropped.resize((256, 256), Image.BICUBIC)

In [ ]:
resized.show()

In [ ]:
b = get_image_bytes('/home/joel/imagenet/ILSVRC2012_img_train/n02105855/n02105855_321.JPEG')

In [28]:
import exifread
from functools import reduce

In [29]:
corrupted_shards = [169, 302, 346, 468]
for name in reduce(lambda x, y: x+y, [sharded_filenames[i] for i in corrupted_shards]):
    with open(name, 'rb') as f:
        try:
            tags = exifread.process_file(f, strict=True)
        except:
            print(name)

/home/joel/imagenet/ILSVRC2012_img_train/n02514041/n02514041_7784.JPEG
/home/joel/imagenet/ILSVRC2012_img_train/n02090379/n02090379_1696.JPEG


Possibly corrupted field UserComment in EXIF IFD
Possibly corrupted field LightingType in MakerNote IFD
Possibly corrupted field NoiseReduction in MakerNote IFD
Possibly corrupted field SoftwareRelease in MakerNote IFD
Possibly corrupted field ImageDescription in Image IFD
Possibly corrupted field Software in Image IFD
Possibly corrupted field ISOSelection in MakerNote IFD
Possibly corrupted field SoftwareRelease in MakerNote IFD
Possibly corrupted field SoftwareRelease in MakerNote IFD
Possibly corrupted field SerialNumber in MakerNote IFD
Possibly corrupted field OwnerName in MakerNote IFD


/home/joel/imagenet/ILSVRC2012_img_train/n02782093/n02782093_4169.JPEG
/home/joel/imagenet/ILSVRC2012_img_train/n01873310/n01873310_30386.JPEG


Possibly corrupted field ISOSelection in MakerNote IFD
Possibly corrupted field Copyright in Image IFD
Possibly corrupted field PictureInfo in MakerNote IFD
Possibly corrupted field Software in Image IFD
Possibly corrupted field LightingType in MakerNote IFD


/home/joel/imagenet/ILSVRC2012_img_train/n03874293/n03874293_4803.JPEG
/home/joel/imagenet/ILSVRC2012_img_train/n03991062/n03991062_19073.JPEG


In [42]:
import xml.etree.ElementTree as ET

In [43]:
val_imgs = Path('/home/joel/imagenet/ILSVRC2012_img_val')
val_labels = Path('/home/joel/imagenet/ILSVRC2012_bbox_val_v3/val')

In [50]:
img_fnames = sorted(val_imgs.iterdir())
label_fnames = sorted(val_labels.iterdir())

In [51]:
len(img_fnames) == len(label_fnames)

True

In [52]:
fname_label = list(zip(img_fnames, label_fnames))

In [48]:
fname_label = list(fname_label)

In [53]:
fname_label[:3]

[(PosixPath('/home/joel/imagenet/ILSVRC2012_img_val/ILSVRC2012_val_00000001.JPEG'),
  PosixPath('/home/joel/imagenet/ILSVRC2012_bbox_val_v3/val/ILSVRC2012_val_00000001.xml')),
 (PosixPath('/home/joel/imagenet/ILSVRC2012_img_val/ILSVRC2012_val_00000002.JPEG'),
  PosixPath('/home/joel/imagenet/ILSVRC2012_bbox_val_v3/val/ILSVRC2012_val_00000002.xml')),
 (PosixPath('/home/joel/imagenet/ILSVRC2012_img_val/ILSVRC2012_val_00000003.JPEG'),
  PosixPath('/home/joel/imagenet/ILSVRC2012_bbox_val_v3/val/ILSVRC2012_val_00000003.xml'))]

In [56]:
i = 0
for img, label in tqdm_notebook(fname_label):
    if img.parts[-1].lstrip('ILSVRC2012_val_').rstrip('.JPEG') != label.parts[-1].lstrip('ILSVRC2012_val_').rstrip('.xml'):
        print('oh no')
    
    if i % 10000 == 0:
        print(img)
        print(label)
        print()
    
    i += 1

/home/joel/imagenet/ILSVRC2012_img_val/ILSVRC2012_val_00000001.JPEG
/home/joel/imagenet/ILSVRC2012_bbox_val_v3/val/ILSVRC2012_val_00000001.xml

/home/joel/imagenet/ILSVRC2012_img_val/ILSVRC2012_val_00010001.JPEG
/home/joel/imagenet/ILSVRC2012_bbox_val_v3/val/ILSVRC2012_val_00010001.xml

/home/joel/imagenet/ILSVRC2012_img_val/ILSVRC2012_val_00020001.JPEG
/home/joel/imagenet/ILSVRC2012_bbox_val_v3/val/ILSVRC2012_val_00020001.xml

/home/joel/imagenet/ILSVRC2012_img_val/ILSVRC2012_val_00030001.JPEG
/home/joel/imagenet/ILSVRC2012_bbox_val_v3/val/ILSVRC2012_val_00030001.xml

/home/joel/imagenet/ILSVRC2012_img_val/ILSVRC2012_val_00040001.JPEG
/home/joel/imagenet/ILSVRC2012_bbox_val_v3/val/ILSVRC2012_val_00040001.xml




In [70]:
# for img, label in tqdm_notebook(fname_label):
#     tree = ET.parse(label)
#     root = tree.getroot()
#     synset = root.find("./object/name").text
#     assert synset in code_to_labels

random.shuffle(fname_label)
imagenet_filenames = fname_label
sharded_filenames = []
shard_size = len(imagenet_filenames) // 128 + 1
assert (shard_size * 128 >= len(imagenet_filenames) and (shard_size-1) * 128 < len(imagenet_filenames))
for i in range(0, len(imagenet_filenames), shard_size):
    sharded_filenames.append(imagenet_filenames[i:i+shard_size])

In [71]:
len(imagenet_filenames)

50000

In [72]:
len(sharded_filenames)

128

In [73]:
sum(len(x) for x in sharded_filenames)

50000

In [75]:
def get_image_bytes(filename):
    im = Image.open(filename)
    x, y = im.size
    if x > y:
        cropped = im.crop(((x-y) // 2, 0,(x+y) // 2, y))
    elif x < y:
        cropped = im.crop((0, (y-x) // 2, x, (y+x) // 2))
    else:
        cropped = im
    
    im = cropped.resize((256, 256), Image.BICUBIC)
    im_bytes = io.BytesIO()
    im.save(im_bytes, format='JPEG')
    return im_bytes.getvalue()

def to_example(encoded, filename, label, synset, text):
    feature_dict = {
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename])),
        'image/class/label' : tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
        'image/class/synset' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[synset])),
        'image/class/text' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[text])),
    }
    
    return tf.train.Example(features=tf.train.Features(feature=feature_dict))

train_tfrecord_path = '/home/joel/imagenet/tfrecords/val-{:04}-0128'
for i, names in tqdm_notebook(enumerate(sharded_filenames), desc='shards'):
    writer = tf.python_io.TFRecordWriter(train_tfrecord_path.format(i))
    for img_label in tqdm_notebook(names, desc='shard {}'.format(i)):
        name = img_label[0]
        label = img_label[1]
        tree = ET.parse(label)
        root = tree.getroot()
        synset = root.find("./object/name").text
        encoded = get_image_bytes(name)
        filename = bytes(name.parts[-1], 'utf-8')
        label, text = code_to_labels[synset]
        synset = bytes(synset, 'utf-8')
        text = bytes(text, 'utf-8')
        writer.write(to_example(encoded, filename, label, synset, text).SerializeToString())
    
    writer.close()
    